In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('indonesian')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.corpus.stopwords.words('indonesian')

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
import numpy as np
import pandas as pd
from sklearn.model_selection import  train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import  accuracy_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Error loading indonesian: Package 'indonesian' not found
[nltk_data]     in index
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
#!dir

In [2]:
df = pd.read_csv("train_preprocess.tsv.txt", sep="\t", engine="python", names=["data", "label"])


In [3]:
df

,data,label
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
...,...,...
10995,tidak kecewa,positive
10996,enak rasa masakan nya apalagi kepiting yang me...,positive
10997,hormati partai-partai yang telah berkoalisi,neutral
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative


In [4]:
df.shape

(11000, 2)

In [5]:
def cleansing(text):
    # Make sentence being lowercase
    text = text.lower()

    # Remove user, rt, \n, retweet, \t, url, xd
    pattern_1 = r'(user|retweet|\\t|\\r|url|xd)'
    text = re.sub(pattern_1, '', text)

    # Remove mention
    pattern_2 = r'@[^\s]+'
    text = re.sub(pattern_2, '', text)

    # Remove hashtag
    pattern_3 = r'#([^\s]+)'
    text = re.sub(pattern_3, '', text)

    # Remove general punctuation, math operation char, etc.
    pattern_4 = r'[\,\@\*\_\-\!\:\;\?\'\.\"\)\(\{\}\<\>\+\%\$\^\#\/\`\~\|\&\|]'
    text = re.sub(pattern_4, ' ', text)

    # Remove single character
    pattern_5 = r'\b\w{1,3}\b'
    text = re.sub(pattern_5, '', text)

    # Remove emoji
    pattern_6 = r'\\[a-z0-9]{1,5}'
    text = re.sub(pattern_6, '', text)

    # Remove digit character
    pattern_7 = r'\d+'
    text = re.sub(pattern_7, '', text)

    # Remove url start with http or https
    pattern_8 = r'(https|https:)'
    text = re.sub(pattern_8, '', text)

    # Remove (\); ([); (])
    pattern_9 = r'[\\\]\[]'
    text = re.sub(pattern_9, '', text)

    # Remove character non ASCII
    pattern_10 = r'[^\x00-\x7f]'
    text = re.sub(pattern_10, '', text)

    # Remove character non ASCII
    pattern_11 = r'(\\u[0-9A-Fa-f]+)'
    text = re.sub(pattern_11, '', text)

    # Remove multiple whitespace
    pattern_12 = r'(\s+|\\n)'
    text = re.sub(pattern_12, ' ', text)
    
    # Remove whitespace at the first and end sentences
    text = text.rstrip()
    text = text.lstrip()
    return text

indo_stop_words = stopwords.words("indonesian")

def remove_stopwords(text):
    return ' '.join([word for word in word_tokenize(text) if word not in indo_stop_words])
  
def tokenisasi(text):
  tokens = nltk.tokenize.word_tokenize(text)
  return tokens

In [6]:
df_coba = df.applymap(cleansing)
df_coba = df_coba.applymap(remove_stopwords)
df_coba['word_token'] = df_coba['data'].apply(tokenisasi)

In [7]:
df_coba

,data,label,word_token
0,warung dimiliki pengusaha pabrik puluhan terke...,positive,"[warung, dimiliki, pengusaha, pabrik, puluhan,..."
1,mohon ulama lurus k mmbri hujjah partai diwlh ...,neutral,"[mohon, ulama, lurus, k, mmbri, hujjah, partai..."
2,lokasi strategis jalan sumatera bandung nyaman...,positive,"[lokasi, strategis, jalan, sumatera, bandung, ..."
3,betapa bahagia unboxing paket barang bagus men...,positive,"[betapa, bahagia, unboxing, paket, barang, bag..."
4,mahasiswa sombong kasih kartu kuning belajar u...,negative,"[mahasiswa, sombong, kasih, kartu, kuning, bel..."
...,...,...,...
10995,kecewa,positive,[kecewa]
10996,enak masakan kepiting menyenangkan memilih kep...,positive,"[enak, masakan, kepiting, menyenangkan, memili..."
10997,hormati partai partai berkoalisi,neutral,"[hormati, partai, partai, berkoalisi]"
10998,pagi pagi pasteur macet parah bikin jengkel,negative,"[pagi, pagi, pasteur, macet, parah, bikin, jen..."


In [8]:
# membuat dictionary mapping
mapping = {'positive': 1, 'neutral': 0, 'negative': 2}

In [9]:
# mengubah nilai pada kolom 'label' dengan method apply
#df_coba['label'] = df_coba['label'].apply(lambda x: [mapping[i] for i in x]) --> hanya jika isi kolom 'label' berbentuk list atau array
df_coba['label'] = df_coba['label'].map(mapping)

In [10]:
df_coba

,data,label,word_token
0,warung dimiliki pengusaha pabrik puluhan terke...,1,"[warung, dimiliki, pengusaha, pabrik, puluhan,..."
1,mohon ulama lurus k mmbri hujjah partai diwlh ...,0,"[mohon, ulama, lurus, k, mmbri, hujjah, partai..."
2,lokasi strategis jalan sumatera bandung nyaman...,1,"[lokasi, strategis, jalan, sumatera, bandung, ..."
3,betapa bahagia unboxing paket barang bagus men...,1,"[betapa, bahagia, unboxing, paket, barang, bag..."
4,mahasiswa sombong kasih kartu kuning belajar u...,2,"[mahasiswa, sombong, kasih, kartu, kuning, bel..."
...,...,...,...
10995,kecewa,1,[kecewa]
10996,enak masakan kepiting menyenangkan memilih kep...,1,"[enak, masakan, kepiting, menyenangkan, memili..."
10997,hormati partai partai berkoalisi,0,"[hormati, partai, partai, berkoalisi]"
10998,pagi pagi pasteur macet parah bikin jengkel,2,"[pagi, pagi, pasteur, macet, parah, bikin, jen..."


In [15]:
# mengubah nilai pada kolom 'label' menjadi integer
#df_coba['label'] = df_coba['label'].apply(lambda x: x[0]).astype(int)

In [11]:
df_coba.isna().sum()

data          0
label         0
word_token    0
dtype: int64

In [12]:
# Menggabungkan setiap elemen dalam kolom 'word_tokens' menjadi satu string
df_coba['string_token'] = df_coba['word_token'].apply(' '.join)

In [13]:
df_coba

,data,label,word_token,string_token
0,warung dimiliki pengusaha pabrik puluhan terke...,1,"[warung, dimiliki, pengusaha, pabrik, puluhan,...",warung dimiliki pengusaha pabrik puluhan terke...
1,mohon ulama lurus k mmbri hujjah partai diwlh ...,0,"[mohon, ulama, lurus, k, mmbri, hujjah, partai...",mohon ulama lurus k mmbri hujjah partai diwlh ...
2,lokasi strategis jalan sumatera bandung nyaman...,1,"[lokasi, strategis, jalan, sumatera, bandung, ...",lokasi strategis jalan sumatera bandung nyaman...
3,betapa bahagia unboxing paket barang bagus men...,1,"[betapa, bahagia, unboxing, paket, barang, bag...",betapa bahagia unboxing paket barang bagus men...
4,mahasiswa sombong kasih kartu kuning belajar u...,2,"[mahasiswa, sombong, kasih, kartu, kuning, bel...",mahasiswa sombong kasih kartu kuning belajar u...
...,...,...,...,...
10995,kecewa,1,[kecewa],kecewa
10996,enak masakan kepiting menyenangkan memilih kep...,1,"[enak, masakan, kepiting, menyenangkan, memili...",enak masakan kepiting menyenangkan memilih kep...
10997,hormati partai partai berkoalisi,0,"[hormati, partai, partai, berkoalisi]",hormati partai partai berkoalisi
10998,pagi pagi pasteur macet parah bikin jengkel,2,"[pagi, pagi, pasteur, macet, parah, bikin, jen...",pagi pagi pasteur macet parah bikin jengkel


In [14]:
text = df_coba['word_token'].tolist()
print(len(text))

11000


In [21]:
token = Tokenizer()
token.fit_on_texts(text)

In [24]:
df_coba['word_to_index'] = token.texts_to_sequences(text)

In [25]:
df_coba

,data,label,word_token,string_token,word_to_index
0,warung dimiliki pengusaha pabrik puluhan terke...,1,"[warung, dimiliki, pengusaha, pabrik, puluhan,...",warung dimiliki pengusaha pabrik puluhan terke...,"[110, 2051, 3086, 2750, 958, 114, 337, 6, 461,..."
1,mohon ulama lurus k mmbri hujjah partai diwlh ...,0,"[mohon, ulama, lurus, k, mmbri, hujjah, partai...",mohon ulama lurus k mmbri hujjah partai diwlh ...,"[559, 711, 2445, 1455, 7567, 7568, 145, 7569, ..."
2,lokasi strategis jalan sumatera bandung nyaman...,1,"[lokasi, strategis, jalan, sumatera, bandung, ...",lokasi strategis jalan sumatera bandung nyaman...,"[28, 121, 12, 1552, 6, 11, 547, 134, 5369, 1, ..."
3,betapa bahagia unboxing paket barang bagus men...,1,"[betapa, bahagia, unboxing, paket, barang, bag...",betapa bahagia unboxing paket barang bagus men...,"[3555, 1157, 7570, 164, 169, 17, 4247, 90]"
4,mahasiswa sombong kasih kartu kuning belajar u...,2,"[mahasiswa, sombong, kasih, kartu, kuning, bel...",mahasiswa sombong kasih kartu kuning belajar u...,"[252, 1020, 127, 353, 734, 1021, 7571, 319, 63..."
...,...,...,...,...,...
10995,kecewa,1,[kecewa],kecewa,[122]
10996,enak masakan kepiting menyenangkan memilih kep...,1,"[enak, masakan, kepiting, menyenangkan, memili...",enak masakan kepiting menyenangkan memilih kep...,"[1, 41, 197, 139, 113, 197, 70, 117, 6]"
10997,hormati partai partai berkoalisi,0,"[hormati, partai, partai, berkoalisi]",hormati partai partai berkoalisi,"[5323, 145, 145, 4080]"
10998,pagi pagi pasteur macet parah bikin jengkel,2,"[pagi, pagi, pasteur, macet, parah, bikin, jen...",pagi pagi pasteur macet parah bikin jengkel,"[152, 152, 827, 196, 387, 56, 2337]"


In [26]:
df_copy = df_coba.copy()
df_copy.drop(columns=["data", "word_token", "string_token"], inplace=True)

In [27]:
df_copy

,label,word_to_index
0,1,"[110, 2051, 3086, 2750, 958, 114, 337, 6, 461,..."
1,0,"[559, 711, 2445, 1455, 7567, 7568, 145, 7569, ..."
2,1,"[28, 121, 12, 1552, 6, 11, 547, 134, 5369, 1, ..."
3,1,"[3555, 1157, 7570, 164, 169, 17, 4247, 90]"
4,2,"[252, 1020, 127, 353, 734, 1021, 7571, 319, 63..."
...,...,...
10995,1,[122]
10996,1,"[1, 41, 197, 139, 113, 197, 70, 117, 6]"
10997,0,"[5323, 145, 145, 4080]"
10998,2,"[152, 152, 827, 196, 387, 56, 2337]"


In [36]:
x = df_copy.iloc[:,1] # parameter (features)
y = df_copy.iloc[:,0] # targeted

In [41]:
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.7)


In [45]:
print(x_train.shape)  # Output: (n_data_points, n_features)
print(y_train.shape)  # Output: (n_data_points,)


(7699,)
(7699,)


In [46]:
x_train_dtype = np.array(x_train).dtype
print(x_train_dtype)

object


In [47]:
y_train_dtype = np.array(y_train).dtype
print(y_train_dtype)

int64


In [49]:
MAX_SEQUENCE_LENGTH = 100

# Lakukan padding pada setiap data input
x_train = pad_sequences(x_train, maxlen=MAX_SEQUENCE_LENGTH)


In [54]:
MAX_SEQUENCE_LENGTH = 100

# Lakukan padding pada setiap data input
x_test = pad_sequences(x_test, maxlen=MAX_SEQUENCE_LENGTH)


In [50]:
x_train = np.array(x_train, dtype=np.int64)


In [58]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 25, 10), random_state=1, max_iter=500)

clf.fit(x_train, y_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(50, 25, 10), max_iter=500,
              random_state=1, solver='lbfgs')

In [61]:
y_pred = clf.predict(x_test)
y_pred # sbetulnya masih termasuk error ini 

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [62]:
accuracy_score(y_test, y_pred)

0.5767949106331415